In [ ]:
# File location and type
file_location = "/FileStore/tables/JPMoptions.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# display(df)

df_op = df.toPandas()
import pandas as pd
df_op['date'] = pd.to_datetime(df_op['date'].astype(str))
df_op['exdate'] = pd.to_datetime(df_op['exdate'].astype(str))
df_op.head()

###### Stock data next 

# File location and type
file_location2 = "/FileStore/tables/JPMstock.csv"
file_type2 = "csv"

# CSV options
infer_schema2 = "true"
first_row_is_header2 = "true"
delimiter2 = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df2 = spark.read.format(file_type2) \
  .option("inferSchema", infer_schema2) \
  .option("header", first_row_is_header2) \
  .option("sep", delimiter2) \
  .load(file_location2)

# display(df)

df_s = df2.toPandas()
import pandas as pd
df_s['date'] = pd.to_datetime(df_s['date'].astype(str))
df_s['close'] = df_s['close'].astype(float)
df_s.head()



In [ ]:
df_op['strike_price'] = df_op['strike_price'].astype(float)
df_op['best_bid'] = df_op['best_bid'].astype(float)
df_op['best_offer'] = df_op['best_offer'].astype(float)
df_op['strike_price'] *= .001
df_op['mid'] = df_op['best_bid']+df_op['best_offer'] 
df_op['mid'] *= 0.5
df_op.head()



In [ ]:
df_op2 = df_op
df_s2 = df_s

df_op2= df_op.set_index(['date', 'exdate'])
df_s2 = df_s2.set_index(df_s['date'])

opdates1 = df_op2.index.get_level_values('date')
expdates1 = df_op2.index.get_level_values('exdate')
stockdates1 = df_s2.index.get_level_values('date')

stockdate_end = stockdates1[-1]
print(stockdate_end)

In [ ]:
### Delete rows w ExpDate or regular date outside of stock data range

df_op3 = df_op2
# stockdates1 = df_s2.index.get_level_values('date')
# stockdate_end = stockdates1[-1]
df_op3 = df_op3[df_op3.index.get_level_values('exdate')<= stockdate_end]
df_op3 = df_op3[df_op3.index.get_level_values('date')<= stockdate_end]

print(df_op3.head())

In [ ]:
expdates2 = list(df_op3.index.get_level_values('exdate'))
stockdates2 = list(df_s2.index.get_level_values('date'))

import numpy as np
main_list = np.setdiff1d(expdates2,stockdates2)
print(main_list) # proves that there are some expDates that are in the stock data range but we dont have specific values for. 

combined_dates= list(set(stockdates2) | set(expdates2))
combined_dates.sort()

print(combined_dates)

df_s3 = df_s2
df_s3 = df_s3.reindex(combined_dates, method = 'bfill')
print(df_s3.head(100))

In [ ]:
### Add stock_close_cur
df_op4 = df_op3
opdates2 = df_op4.index.get_level_values('date')
expdates2 = df_op4.index.get_level_values('exdate')
df_op4 = df_op4.reset_index()
df_op4 = df_op4.set_index(['date'])
merged = df_op4.merge(df_s3[['close']], on='date')
merged.rename(columns={'close':'stock_close_cur'}, inplace=True)
merged = merged.reset_index()
merged = merged.set_index(['date','exdate'])
merged.head()

In [ ]:
# Cleaning stock dataframe
df_s4 = df_s3
df_s4 = df_s4.drop(['date'], axis=1)
df_s4 = df_s4.reset_index()
df_s4.rename(columns={'date':'exdate'}, inplace=True)
df_s4 = df_s4.set_index(['exdate'])
df_s4.head()

In [ ]:
### Add stock_close_exp

merged = merged.reset_index()
merged = merged.set_index(['exdate'])
merged = merged.merge(df_s4[['close']], left_index=True, right_index=True)
merged.rename(columns={'close':'stock_close_exp'}, inplace=True)

pd.set_option('display.max_rows', None)
merged = merged.reset_index()
merged = merged.set_index(['date','exdate'])
merged.sort_index(inplace=True)
merged.head(40)



In [ ]:
import numpy as np
df_op5 = merged
df_op5.head()
df_op5['DaysToExp'] = (df_op5.index.get_level_values('exdate') - df_op5.index.get_level_values('date')).days
df_op5['StockMoveByExp'] = (df_op5['stock_close_exp'] / df_op5['stock_close_cur']) -1 
df_op5['BreakEvenDist'] = ((df_op5['mid'] + df_op5['strike_price']) / df_op5['stock_close_cur']) - 1
# df_op5['OpRetByExp'] = (df_op5['stock_close_exp'] - (df_op5['mid'] + df_op5['strike_price'])) / df_op5['mid']
df_op6 = df_op5

def OpRet(row):
  if row['stock_close_exp'] - (row['strike_price']) > 0:
    return (row['stock_close_exp'] - (row['mid'] + row['strike_price'])) / row['mid']
  elif row['stock_close_exp'] - (row['strike_price']) < 0:
    return -1

df_op6['OpRetByExp'] = df_op6.apply (lambda row: OpRet(row), axis=1)

df_op6.head(30)



In [ ]:
df_op7 = df_op6

def OpRisk(row):
  if row['stock_close_cur']*(1-0.1) -  row['strike_price'] > 0:
    return (row['stock_close_cur']*(1-0.1) - (row['mid'] + row['strike_price'])) / row['mid']
  elif row['stock_close_cur']*(1-0.1) -  row['strike_price'] < 0:
    return -1

df_op7['OpLossIfStockFall10'] = df_op7.apply(lambda row: OpRisk(row), axis=1)

In [ ]:
df_op8 = df_op7[df_op7['cp_flag']=='C']
df_op8['OpRiskAdjRet'] = (-0.1 / df_op8['OpLossIfStockFall10']) * df_op8['OpRetByExp']
df_op8['Moneyness'] = (df_op8['stock_close_cur'] - df_op8['strike_price']) / df_op8['stock_close_cur']
df_op8['FutRetVsBreakeven'] = df_op8['StockMoveByExp'] - df_op8['BreakEvenDist'] 
df_op8.head(50)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(df_op8['Moneyness'], df_op8['FutRetVsBreakeven'], 'o', color='black')
display(fig)
fig2, ax2 = plt.subplots()
ax2.plot(df_op8['impl_volatility'], df_op8['FutRetVsBreakeven'], 'o', color='red')
display(fig2)

fig3, ax3 = plt.subplots()
ax3.plot(df_op8['Moneyness'], df_op8['OpRetByExp'], 'o', color='yellow')
display(fig3)

fig4, ax4 = plt.subplots()
ax4.plot(df_op8['impl_volatility'], df_op8['OpRetByExp'], 'o', color='green')
display(fig4)

fig5, ax5 = plt.subplots()
ax5.plot(df_op8['Moneyness'], df_op8['OpRiskAdjRet'], 'o', color='orange')
display(fig5)

fig6, ax6 = plt.subplots()
ax6.plot(df_op8['impl_volatility'], df_op8['OpRiskAdjRet'], 'o', color='purple')
display(fig6)